# Actividad 3 | Aprendizaje supervisado y no supervisado

**Alumna:** Alejandra Berenice Vega Lopez
**Matricula:** A01795415
  
### Objetivo:
Aplicar algoritmos de aprendizaje supervisado y no supervisado mediante PySpark para la resolución de problemas en análisis de datos, fomentando el desarrollo de habilidades prácticas en el manejo y procesamiento eficiente de grandes conjuntos de datos.
 
### Instrucciones:
En esta actividad aprenderás a aplicar algoritmos de aprendizaje supervisado y no supervisado implementados en PySpark, identificando para ello cada una de las etapas necesarias para poder ejecutar con éxito dichos algoritmos. En particular, se espera que apliques con éxito un algoritmo de aprendizaje supervisado (DecisionTree, RandomForest, GBTClassifier, Multilayer Perceptron, entre otros) y uno de aprendizaje no supervisado (K means, GaussianMixture, PIC, entre otros) a partir de las implementaciones disponibles en PySpark, aplicando las etapas necesarias y suficientes de preparación de los datos de entrada.

Para lograrlo, se sugiere seguir los siguientes pasos:


### 1. Introducción teórica: 
Se deberá de documentar de forma breve los conceptos de aprendizaje supervisado, no supervisado, así como los algoritmos más representativos que se identifican en la literatura de cada tipo de aprendizaje, además de identificar aquellos que están disponible a través de PySpark.


**¿Qué es el Aprendizaje Automático?**
El aprendizaje automático (machine learning) es una rama de la inteligencia artificial que permite a los sistemas aprender automáticamente a partir de datos, identificar patrones y tomar decisiones sin estar explícitamente programados para cada tarea. Dentro de esta disciplina, los enfoques más comunes se dividen en dos grandes grupos: aprendizaje supervisado y aprendizaje no supervisado.

**Aprendizaje Supervisado**
El aprendizaje supervisado se basa en entrenar modelos con datos etiquetados, es decir, donde cada instancia de entrada está asociada a una salida o clase esperada. El objetivo es que el modelo aprenda una función que relacione las características de entrada con la etiqueta y pueda generalizar para nuevos casos. PySpark implementa estos algoritmos a través del módulo pyspark.ml.classification, que forma parte del paquete MLlib. Este módulo permite construir pipelines con etapas encadenadas de transformación y modelado. Algunos de los algoritmos representativos son los siguientes:
- Árboles de Decisión (Decision Tree): modelos jerárquicos para dividir el espacio de características.
- Random Forest: ensambles de árboles para mejorar precisión y reducir sobreajuste.
- Gradient Boosted Trees (GBTClassifier): optimiza secuencialmente árboles para mejorar predicción.
- Multilayer Perceptron (MLPClassifier): red neuronal de múltiples capas usada en problemas no lineales.


**Aprendizaje No Supervisado**
El aprendizaje no supervisado trabaja con datos sin etiquetas. El objetivo es descubrir patrones latentes o estructuras ocultas como agrupamientos o relaciones internas. Estos algoritmos están disponibles en el módulo pyspark.ml.clustering, que permite aplicarlos sobre grandes volúmenes de datos distribuidos. Algunos de los algoritmos representativos son los siguientes:
- K-Means: agrupamiento según distancia a centroides.
- Gaussian Mixture Model (GMM): mezcla probabilística de distribuciones gaussianas.
- Power Iteration Clustering (PIC): técnica espectral eficiente para grandes datasets.


**Preparación de Datos en PySpark**
Para ambos tipos de aprendizaje, se requiere una etapa previa de preparación de datos, la cual incluye algunos de los siguientes ejemplos, dependiendo de las transformaciones necesarias para cada Dataset:
- Limpieza de datos y tratamiento de valores faltantes
- Codificación de variables categóricas (StringIndexer, OneHotEncoder)
- Ensamblaje de columnas (VectorAssembler)
- Escalado de características (StandardScaler), entre otros.


**Bibloografía:**
Polak, A. (2023). Scaling machine learning with SparkLinks to an external site.: distributed ML with MLlib, TensorFlow, and PyTorch. " O'Reilly Media, Inc."., Chapter 4: Data Ingestión, Preprocessing and Descriptive Statistics
Apache Spark. (2023). MLlib: Classification. https://spark.apache.org/docs/latest/ml-classification-regression.html


### 2. Selección de los datos: 

Para esta actividad, se propone recolectar una muestra de dimensión contenida (para evitar que los tiempos de procesamiento sean altos) a partir de la base de datos que estás trabajando en tu proyecto. Para ello y tomando como base la actividad previa en la cual has implementado códigos que permiten obtener particiones de la base de datos global D que cumplen con los criterios de las variables de caracterización identificadas, se propone que recuperes un número limitado de instancias de cada partición (aplicando la técnica de muestreo que propusiste en el Módulo 3, Proyecto Base de datos de Big Data, paso 4), lo que te permitirá construir una muestra M a partir de la unión de las instancias que se recuperan de este proceso.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, split, count
from pyspark.sql.types import FloatType, IntegerType

spark = SparkSession.builder.appName('MuestreoBooksRating').getOrCreate()

# Cargar dataset
df = spark.read.option('header', 'true').csv('Books_rating.csv')
df.printSchema()
df.select('review/score', 'review/helpfulness', 'user_id').show(5) ##Ejemplo de variables de participacion

25/05/25 15:40:50 WARN Utils: Your hostname, MacBook-Pro-de-Juan.local resolves to a loopback address: 127.0.0.1; using 192.168.100.17 instead (on interface en0)
25/05/25 15:40:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/25 15:40:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

+------------+------------------+--------------+
|review/score|review/helpfulness|       user_id|
+------------+------------------+--------------+
|         4.0|               7/7| AVCGYZL8FQQTD|
|         5.0|             10/10|A30TK6U7DNS82R|
|         5.0|             10/11|A3UH4UZ4RSVO82|
|         4.0|               7/7|A2MVUWT453QH61|
|         4.0|               3/3|A22X4XUPKF66MR|
+------------+------------------+--------------+
only showing top 5 rows



In [2]:
# Remover advertencias
spark.sparkContext.setLogLevel("ERROR")

In [3]:
# Limpieza de datos 
df_clean = df.filter((col("review/score").isNotNull()) & 
                     (col("review/helpfulness").isNotNull()) & 
                     (col("user_id").isNotNull()))
df = df_clean

# Convertir columnas relevantes a int
df = df.withColumn("Score_num", col("review/score").cast(FloatType()))
df = df.withColumn("Helpfulness_num", split(col("review/helpfulness"), "/")[0].cast(IntegerType()))

# Clasificar score
df = df.withColumn("score_group", when(col("Score_num") >= 4, "Alta").otherwise("Baja"))
# Clasificar helpfulness
df = df.withColumn("helpfulness_group", when(col("Helpfulness_num") >= 8, "Alta").otherwise("Baja"))

# Remover valores no convertidos correctamente
df = df.filter((col("Score_num").isNotNull()) & (col("Helpfulness_num").isNotNull()))

# Calcular cantidad de reseñas por usuario
user_reviews = df.groupBy("user_id").agg(count("user_id").alias("review_count"))
df = df.join(user_reviews, on="user_id", how="left")

# Clasificación más detallada de usuarios según frecuencia de participación
df = df.withColumn(
    "user_group_detailed",
    when(col("review_count") >= 20, "20+ reseñas")
    .when((col("review_count") >= 10) & (col("review_count") < 20), "10-19 reseñas")
    .when((col("review_count") >= 5) & (col("review_count") < 10), "5-9 reseñas")
    .when((col("review_count") >= 2) & (col("review_count") < 5), "2-4 reseñas")
    .otherwise("1 reseña")
)

In [4]:
# Mostrar distribución con mejor formato
from pyspark.sql.functions import format_number
distribution = df.groupBy("user_group_detailed").count().orderBy("count", ascending=False)
distribution = distribution.withColumn("count", format_number(col("count"), 0))
distribution.show(truncate=False, n=100)

[Stage 6:====================================================>    (12 + 1) / 13]

+-------------------+-------+
|user_group_detailed|count  |
+-------------------+-------+
|1 reseña           |690,877|
|2-4 reseñas        |580,802|
|20+ reseñas        |540,951|
|5-9 reseñas        |338,868|
|10-19 reseñas      |268,931|
+-------------------+-------+



In [5]:
# Revisión de distribución conjunta de variables de caracterización
df.groupBy("score_group", "helpfulness_group", "user_group_detailed").count().orderBy("count", ascending=False).show()

[Stage 15:===========================================>            (10 + 3) / 13]

+-----------+-----------------+-------------------+------+
|score_group|helpfulness_group|user_group_detailed| count|
+-----------+-----------------+-------------------+------+
|       Alta|             Baja|           1 reseña|482065|
|       Alta|             Baja|        2-4 reseñas|402632|
|       Alta|             Baja|        20+ reseñas|356614|
|       Alta|             Baja|        5-9 reseñas|243589|
|       Alta|             Baja|      10-19 reseñas|190730|
|       Baja|             Baja|           1 reseña|104137|
|       Baja|             Baja|        2-4 reseñas| 97797|
|       Alta|             Alta|        20+ reseñas| 85625|
|       Baja|             Baja|        20+ reseñas| 78467|
|       Alta|             Alta|           1 reseña| 72828|
|       Baja|             Baja|        5-9 reseñas| 53827|
|       Alta|             Alta|        2-4 reseñas| 53819|
|       Baja|             Baja|      10-19 reseñas| 44771|
|       Baja|             Alta|           1 reseña| 3184

In [6]:
# Cálculo de ocurrencias por combinación
comb_counts = df.groupBy("score_group", "helpfulness_group", "user_group_detailed").count()
total = df.count()
comb_probs = comb_counts.withColumn("probabilidad", (col("count") / total))
comb_probs.orderBy(col("probabilidad").desc()).show(truncate=False)

[Stage 27:========>                                               (2 + 11) / 13]

+-----------+-----------------+-------------------+------+--------------------+
|score_group|helpfulness_group|user_group_detailed|count |probabilidad        |
+-----------+-----------------+-------------------+------+--------------------+
|Alta       |Baja             |1 reseña           |482065|0.1991651066815015  |
|Alta       |Baja             |2-4 reseñas        |402632|0.1663473706520621  |
|Alta       |Baja             |20+ reseñas        |356614|0.14733503854068844 |
|Alta       |Baja             |5-9 reseñas        |243589|0.1006387710608326  |
|Alta       |Baja             |10-19 reseñas      |190730|0.07880008048160057 |
|Baja       |Baja             |1 reseña           |104137|0.04302419116611146 |
|Baja       |Baja             |2-4 reseñas        |97797 |0.0404048207982965  |
|Alta       |Alta             |20+ reseñas        |85625 |0.035375960212012   |
|Baja       |Baja             |20+ reseñas        |78467 |0.03241863322576287 |
|Alta       |Alta             |1 reseña 

In [7]:
# Muestreo por todas las combinaciones posibles (2x2x7 = 28 combinaciones)
from itertools import product
selected_columns = ["user_id", "review/helpfulness", "review/score", "review/summary", "Score_num", "user_group_detailed"]

for s, h in product(["Alta", "Baja"], repeat=2):
    for u in ["20+ reseñas", "10-19 reseñas", "5-9 reseñas", "2-4 reseñas", "1 reseña"]:
        subset = df.filter((col("score_group") == s) &
                           (col("helpfulness_group") == h) &
                           (col("user_group_detailed") == u))
        count_subset = subset.count()
        print(f"Muestra: Score={s}, Helpfulness={h}, Usuario={u}, Registros={count_subset}")
        muestra = subset.select(*selected_columns).sample(False, 0.1, seed=42)
        muestra.show(5, truncate=False)

Muestra: Score=Alta, Helpfulness=Alta, Usuario=20+ reseñas, Registros=85625


+--------------+------------------+------------+------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                            |Score_num|user_group_detailed|
+--------------+------------------+------------+------------------------------------------+---------+-------------------+
|A1075MZNVRMSEO|17/19             |4.0         |A great book about destructive temptations|4.0      |20+ reseñas        |
|A1075MZNVRMSEO|24/24             |5.0         |A great mid-size dictionary               |5.0      |20+ reseñas        |
|A1075MZNVRMSEO|10/10             |5.0         |A wonderful textbook for serious students.|5.0      |20+ reseñas        |
|A11L4SBY7NCSZU|35/36             |5.0         |Fantastic                                 |5.0      |20+ reseñas        |
|A11LNPG39A2ZV4|28/30             |4.0         |Best for Beginners                        |4.0      |20+ reseñas        |
+--------------+--------

Muestra: Score=Alta, Helpfulness=Alta, Usuario=10-19 reseñas, Registros=23537


+--------------+------------------+------------+---------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                               |Score_num|user_group_detailed|
+--------------+------------------+------------+---------------------------------------------+---------+-------------------+
|A109DZXQULGEUK|22/24             |5.0         |A modern classic                             |5.0      |10-19 reseñas      |
|A10FBJXMQPI0LL|10/13             |4.0         |I really enjoyed this book!                  |4.0      |10-19 reseñas      |
|A10MR5DEPRCX98|10/16             |5.0         |FORGET HAMLET, A NEW GUY IS IN TOWN          |5.0      |10-19 reseñas      |
|A142SWWCTIKL0H|42/42             |5.0         |Hard to believe this was written decades ago!|5.0      |10-19 reseñas      |
|A14YHC72SHZHRT|27/27             |4.0         |Delicious food; a couple of minor gripes     |4.0      |10-19 reseñas      |


Muestra: Score=Alta, Helpfulness=Alta, Usuario=5-9 reseñas, Registros=27862


...

Muestra: Score=Baja, Helpfulness=Baja, Usuario=2-4 reseñas, Registros=97797


+--------------+------------------+------------+-------------------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                                         |Score_num|user_group_detailed|
+--------------+------------------+------------+-------------------------------------------------------+---------+-------------------+
|A104EP737ZROHQ|4/10              |1.0         |UGH!!                                                  |1.0      |2-4 reseñas        |
|A105XBOOCZNISQ|7/19              |1.0         |Self published, unreadable, apologia for a child rapist|1.0      |2-4 reseñas        |
|A1086X5YV9M4QF|0/0               |2.0         |Coffin to Heaven                                       |2.0      |2-4 reseñas        |
|A10SJ1NNSQ1CB |3/16              |1.0         |Too much bias to be objective.                         |1.0      |2-4 reseñas        |
|A10W5NFZ9PLX4K|3/10              |1.0         |A pure,

Muestra: Score=Baja, Helpfulness=Baja, Usuario=1 reseña, Registros=104137


[Stage 308:=============================================>         (18 + 4) / 22]

+--------------+------------------+------------+------------------------------------------------------------+---------+-------------------+
|user_id       |review/helpfulness|review/score|review/summary                                              |Score_num|user_group_detailed|
+--------------+------------------+------------+------------------------------------------------------------+---------+-------------------+
|A102LSCF3BSKUR|0/8               |1.0         |New Spring by Robert Jordan                                 |1.0      |1 reseña           |
|A107ZPHSRVHZ00|1/1               |2.0         |Kindle Download/Publisher Proofreading Errors               |2.0      |1 reseña           |
|A109PJ5IB5R6N2|4/6               |3.0         |Out of date content! However, good explanation, good CD-ROM.|3.0      |1 reseña           |
|A10R2Z2FQLPYEY|1/2               |1.0         |kindle version full of typos                                |1.0      |1 reseña           |
|A10Y0YM81YDLIT|1/1 

In [8]:
# Construcción de la muestra M a partir de muestreo estratificado limitado por partición
from itertools import product
user_groups = ["20+ reseñas", "10-19 reseñas", "5-9 reseñas", "2-4 reseñas", "1 reseña"]
selected_columns = ["user_id", "Title", "review/helpfulness", "review/score", "review/summary", "Score_num", "user_group_detailed"]
muestras = []

for s, h in product(["Alta", "Baja"], repeat=2):
    for u in user_groups:
        subset = df.filter((col("score_group") == s) &
                           (col("helpfulness_group") == h) &
                           (col("user_group_detailed") == u))
        muestra = subset.select(*selected_columns).limit(2000)
        muestras.append(muestra)

# Unión de todas las muestras para formar M
from functools import reduce
from pyspark.sql import DataFrame
M = reduce(DataFrame.unionByName, muestras)
print(f"Total de registros en muestra M: {M.count()}")
M.show(10, truncate=False)

Total de registros en muestra M: 40000


[Stage 425:====>                                                  (1 + 12) / 13]

+--------------+-----------------------------------------------------------------------------------------------+------------------+------------+----------------------------------------------------+---------+-------------------+
|user_id       |Title                                                                                          |review/helpfulness|review/score|review/summary                                      |Score_num|user_group_detailed|
+--------------+-----------------------------------------------------------------------------------------------+------------------+------------+----------------------------------------------------+---------+-------------------+
|A1075MZNVRMSEO|El Rey de La Habana (Narrativas Hispanicas)                                                    |8/9               |5.0         |Another reality check about Cuba                    |5.0      |20+ reseñas        |
|A1075MZNVRMSEO|The Ultimate Spanish Review and Practice: Mastering Spanish Grammar for 

**Técnica aplicada:** Muestreo estratificado sobre combinaciones de score, utilidad y tipo de usuario. Se utiliza esta técnica porque permite garantizar que cada subgrupo esté representado proporcionalmente en la muestra.

**Justificación:**
- Los usuarios muy activos pueden dominar el conjunto si no se estratifica.
- Las reseñas útiles y bien puntuadas tienen mayor influencia sobre recomendaciones.
- El análisis cruzado de estas variables representa diferentes comportamientos del lector.

### 3. Preparación de los datos: 

En esta etapa, se deberán de aplicar estrategias de corrección sobre los datos que integran a la muestra M que se ha preparado en el paso previo, de tal forma que de deje un conjunto M listo para ser procesado por los algoritmos de aprendizaje a aplicar. Para ello se deben de considerar pasos como: corrección de registros / columnas con valores nulos, identificación de valores atípicos, transformación de los tipos de datos, etc. Con lo anterior, se tendrá una muestra M pre-procesada.

In [9]:
from pyspark.sql.functions import col, split, length
# Eliminar registros con valores nulos en columnas clave
M_clean = M.dropna(subset=["review/score", "review/helpfulness", "review/summary", "user_id", "Title"])

# Convertir columnas a tipos adecuados si no lo están
M_clean = M_clean.withColumn("Score_num", col("review/score").cast("float"))
M_clean = M_clean.withColumn("Helpfulness_num", split(col("review/helpfulness"), "/")[0].cast("int"))

# Filtrar valores extremos (outliers) por Score_num
M_clean = M_clean.filter((col("Score_num") >= 1.0) & (col("Score_num") <= 5.0))

# Remover textos extremadamente cortos o largos
M_clean = M_clean.filter((length(col("review/summary")) > 3) & (length(col("review/summary")) < 300))

# Mostrar información básica de la muestra limpia
print(f"Número de registros después de limpieza: {M_clean.count()}")
M_clean.select("Score_num", "Helpfulness_num", "review/summary").show(5, truncate=False)

Número de registros después de limpieza: 39870


[Stage 563:>                                                      (0 + 12) / 13]

+---------+---------------+--------------------------------------------------+
|Score_num|Helpfulness_num|review/summary                                    |
+---------+---------------+--------------------------------------------------+
|5.0      |8              |Another reality check about Cuba                  |
|5.0      |145            |Thorough review and great practical exercises     |
|5.0      |10             |A wonderful textbook for serious students.        |
|5.0      |31             |A unique reference that tackles important concepts|
|5.0      |9              |Very helpful resource                             |
+---------+---------------+--------------------------------------------------+
only showing top 5 rows



### 3. Preparación del conjunto de entrenamiento y prueba:

Para esta etapa, la muestra M será divida en un conjunto de entrenamiento y prueba. Para ello, deberás proponer una técnica de muestreo que te permita construir el conjunto de entrenamiento y prueba minimizando el riesgo de inyección de sesgos. Ten en cuenta que, para este punto, deberás de tener en claro el porcentaje de división a utilizar, el cual se deberá de justificar.

**Muestreo aleatorio estratificado proporcional**
Para reducir el riesgo de sesgos, se eligio realizar un muestreo aleatorio estratificado basado en la variable Score_num, ya que representa la etiqueta para tareas de clasificación supervisada y unque PySpark no ofrece directamente una función de muestreo estratificado como en scikit-learn, esto se puede manejar indirectamente al garantizar que la división se haga de forma aleatoria y que la muestra M_clean esté previamente balanceada por partición (lo cual ya se garantizó en la construcción de la muestra M).

**Proporción de división**: 
80% entrenamiento – 20% prueba
La eleccion de este particionamiento es que historicamente proporciones más bajas para entrenamiento (por ejemplo 60%) pueden afectar la capacidad del modelo para aprender correctamente y proporciones mayores (90% entrenamiento) podrían no permitir una validación suficientemente confiable.

In [10]:
# División aleatoria con semilla para reproducibilidad
train_data, test_data = M_clean.randomSplit([0.8, 0.2], seed=42)

# Verificación de tamaños
print(f"Registros de entrenamiento: {train_data.count()}")
print(f"Registros de prueba: {test_data.count()}")

# Validación de proporción de clases (opcional)
from pyspark.sql.functions import count
train_data.groupBy("Score_num").agg(count("*").alias("frecuencia")).orderBy("Score_num").show()
test_data.groupBy("Score_num").agg(count("*").alias("frecuencia")).orderBy("Score_num").show()


Registros de entrenamiento: 32072


Registros de prueba: 7798


+---------+----------+
|Score_num|frecuencia|
+---------+----------+
|      1.0|      5761|
|      2.0|      3774|
|      3.0|      6463|
|      4.0|      3151|
|      5.0|     12923|
+---------+----------+



[Stage 868:>                                                      (0 + 12) / 13]

+---------+----------+
|Score_num|frecuencia|
+---------+----------+
|      1.0|      1412|
|      2.0|       935|
|      3.0|      1569|
|      4.0|       732|
|      5.0|      3150|
+---------+----------+



### 4. Construcción de modelos de aprendizaje supervisado y no supervisado:

Para este punto realizarás dos experimentos separados, dónde se aplicará un algoritmo de aprendizaje supervisado y uno de aprendizaje no supervisado sobre la muestra M. Para el caso de aprendizaje supervisado, se deberá de identificar cuál es la variable objetivo (columna) de aprendizaje, mientras que, para el caso de aprendizaje no supervisado, se debe de seleccionar todas las columnas que se desean considerar como características bajo las cuales se realizará el proceso de agrupamiento. Usando las implementaciones correspondientes de PySpark, se deberá de ejecutar el aprendizaje correspondiente a partir de la invocación de las funciones respectivas. Para este ejercicio, se deberá seleccionar un criterio básico para medir la calidad del resultado obtenido, dependiendo de cada tipo de aprendizaje implementado. La elección quedará a juicio de cada estudiante.

### Modelo de Aprendizaje Supervisado:
Variable objetivo: Score_num
Este campo representa la puntuación que el usuario dio al libro (de 1 a 5) y dado que este es un valor numérico discreto y ordinal, el problema puede abordarse como una clasificación multiclase. El modelo seleccionado para RandomForest ya que se adapta a este ejercicio  al poder manejar clases desequilibradas, es robusto frente a outliers y no requiere normalización de las variables.


**Objetivo del modelo supervisado:**
El propósito de aplicar un algoritmo de aprendizaje supervisado es predecir la calificación (Score_num) que un usuario podría dar a un libro, con base en características conocidas de su reseña. Estas características incluyen la utilidad de la reseña (Helpfulness_num), la longitud del resumen (Length_summary), y el nivel de actividad del usuario (user_group_detailed).

Se ha elegido Score_num como variable objetivo porque es una etiqueta categórica ordinal (valores del 1 al 5), y permite abordar el problema como una clasificación multiclase. El algoritmo RandomForest se selecciona por su robustez frente a outliers, su capacidad para manejar clases desequilibradas, y su facilidad de interpretación.

La salida esperada del modelo es una predicción de Score_num, que nos permite anticipar el tipo de reseña esperada para ciertos perfiles de usuario y contenido. 


**Preparación de datos:**
Seleccionamos como características:
- Helpfulness_num: Las reseñas con mayor utilidad suelen correlacionarse con opiniones más fundamentadas, lo cual puede influir en la puntuación (Score_num).
- Length_summary: La longitud de un resumen puede reflejar el nivel de implicación del usuario.
- user_group_detailed: Usuarios con más reseñas podrían tener un comportamiento de calificación más estable, racional o diverso.

In [11]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import length

# Suponiendo que `train_data` y `test_data` ya están definidos en el entorno
# Agregar columna de longitud del resumen si no existe
train_data = train_data.withColumn("Length_summary", length("review/summary"))
test_data = test_data.withColumn("Length_summary", length("review/summary"))

# Pipeline de características: TF-IDF + ensamblador
tokenizer = Tokenizer(inputCol="review/summary", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1000)
idf = IDF(inputCol="rawFeatures", outputCol="textFeatures")
assembler = VectorAssembler(inputCols=["Helpfulness_num", "Length_summary", "textFeatures"], outputCol="features")

# Clasificador Random Forest
rf = RandomForestClassifier(labelCol="Score_num", featuresCol="features", numTrees=20, seed=42)

# Crear y entrenar pipeline
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, assembler, rf])
modelo_rf = pipeline.fit(train_data)

# Predicciones
predicciones_rf = modelo_rf.transform(test_data)

# Evaluación
evaluator = MulticlassClassificationEvaluator(labelCol="Score_num", predictionCol="prediction", metricName="accuracy")
accuracy_rf = evaluator.evaluate(predicciones_rf)

accuracy_rf


0.40394973070017953

In [12]:
print(f"Exactitud del modelo supervisado: {accuracy_rf:.4f}")

Exactitud del modelo supervisado: 0.4039


### Resultados de modelo Random forest:
El modelo Random Forest arrojó una precisión del 40.39% al predecir la variable objetivo Score_num, que representa la puntuación que un usuario asigna a un libro (del 1 al 5). Este nivel de exactitud indica un desempeño moderado, y aunque está por encima del azar.Este resultado puede deberse a varios factores por ejemplo que la variable Score_num presenta un desequilibrio de clases: las puntuaciones altas (como 5) son mucho más frecuentes que las bajas (como 1 o 2), lo que puede hacer que el modelo favorezca las clases mayoritarias. Además, aunque las variables seleccionadas (Helpfulness_num, Length_summary y user_group_detailed) capturan ciertos aspectos del comportamiento del usuario, podrían no ser suficientes para explicar completamente las diferencias en las calificaciones otorgadas.

### Modelo de Aprendizaje No Supervisado:

En este caso, se optó por utilizar un algoritmo de aprendizaje no supervisado con el fin de explorar patrones ocultos de agrupamiento entre las reseñas, sin tener como referencia una variable objetivo específica. Para ello, se utilizó el algoritmo KMeans, ampliamente reconocido por su eficiencia y simplicidad para dividir instancias en grupos homogéneos en función de sus características.


**Objetivo del modelo no supervisado:**
El propósito de aplicar un algoritmo no supervisado es descubrir patrones ocultos o agrupamientos naturales entre las reseñas de los usuarios, sin necesidad de una etiqueta de salida conocida.

Se ha optado por KMeans ya que es un método eficiente y ampliamente utilizado para segmentar datos numéricos. Las características seleccionadas (Helpfulness_num, Length_summary y user_group_index) permiten representar de forma compacta la intensidad de participación del usuario, su nivel de aporte útil y la extensión del contenido textual.

La salida del modelo es una nueva variable llamada cluster, que indica el grupo asignado a cada reseña. Esta clasificación permite identificar perfiles de usuario como: usuarios casuales, expertos, superusuarios, o perfiles con comportamientos atípicos. Estos hallazgos pueden emplearse para diseñar estrategias de fidelización o mejorar mecanismos de reputación en plataformas de reseñas.


**Preparación de datos:**
Seleccionamos como características:
- Helpfulness_num: Se espera que las reseñas consideradas más útiles reflejen opiniones más fundamentadas, lo que podría generar agrupamientos significativos.
- Length_summary: La longitud del texto puede ser indicativa del nivel de detalle o compromiso del usuario, lo cual también influye en su percepción.
- user_group_detailed: Este campo representa el nivel de experiencia o actividad del usuario en la plataforma (basado en el número de reseñas que ha hecho), lo cual puede ser determinante en sus patrones de comportamiento.


In [14]:
from pyspark.sql.functions import length
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator

# Indexar columna categórica
indexer = StringIndexer(inputCol="user_group_detailed", outputCol="user_group_index")
data_indexed = indexer.fit(M_clean).transform(M_clean)

# Añadir columna de longitud del resumen
data_indexed = data_indexed.withColumn("Length_summary", length("review/summary"))

# Vector de características para clustering
assembler_unsup = VectorAssembler(
    inputCols=["Helpfulness_num", "Length_summary", "user_group_index"],
    outputCol="features"
)
data_unsup = assembler_unsup.transform(data_indexed)

# KMeans clustering
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=5, seed=42)
modelo_kmeans = kmeans.fit(data_unsup)

# Aplicar modelo
resultados_kmeans = modelo_kmeans.transform(data_unsup)

# Evaluación con Silhouette Score
evaluator = ClusteringEvaluator(
    predictionCol="cluster",
    featuresCol="features",
    metricName="silhouette"
)
silhouette_score = evaluator.evaluate(resultados_kmeans)

print(f"Silhouette Score del modelo KMeans: {silhouette_score:.4f}")

# Ejemplos de clusters
resultados_kmeans.select("user_id", "Helpfulness_num", "Length_summary", "user_group_detailed", "cluster").show(10, truncate=False)


Silhouette Score del modelo KMeans: 0.6503


[Stage 2624:=====================>                                 (5 + 8) / 13]

+--------------+---------------+--------------+-------------------+-------+
|user_id       |Helpfulness_num|Length_summary|user_group_detailed|cluster|
+--------------+---------------+--------------+-------------------+-------+
|A1075MZNVRMSEO|8              |32            |20+ reseñas        |0      |
|A1075MZNVRMSEO|145            |45            |20+ reseñas        |1      |
|A1075MZNVRMSEO|10             |42            |20+ reseñas        |2      |
|A1075MZNVRMSEO|31             |50            |20+ reseñas        |4      |
|A1075MZNVRMSEO|9              |21            |20+ reseñas        |0      |
|A1075MZNVRMSEO|9              |21            |20+ reseñas        |0      |
|A1075MZNVRMSEO|30             |44            |20+ reseñas        |4      |
|A1075MZNVRMSEO|17             |42            |20+ reseñas        |2      |
|A1075MZNVRMSEO|14             |41            |20+ reseñas        |2      |
|A1075MZNVRMSEO|17             |52            |20+ reseñas        |2      |
+-----------

In [15]:
from pyspark.sql.functions import avg, count

# Agrupar por clúster
cluster_summary = resultados_kmeans.groupBy("cluster").agg(
    count("*").alias("total_registros"),
    avg("Helpfulness_num").alias("avg_helpfulness"),
    avg("Length_summary").alias("avg_length")
).orderBy("cluster")

# Mostrar resumen por clúster
cluster_summary.show(truncate=False)


[Stage 2680:================>                                      (4 + 9) / 13]

+-------+---------------+------------------+------------------+
|cluster|total_registros|avg_helpfulness   |avg_length        |
+-------+---------------+------------------+------------------+
|0      |23810          |6.0889962200755985|20.408189836203277|
|1      |284            |167.7288732394366 |33.13380281690141 |
|2      |12320          |8.596022727272727 |50.81948051948052 |
|3      |28             |628.5357142857143 |38.67857142857143 |
|4      |3428           |42.56242707117853 |32.02246207701283 |
+-------+---------------+------------------+------------------+



### Resultados de modelo KMeans:
En este ejempplo en especifico se eligió KMeans con un número de clústeres igual a 5, considerando las características numéricas más relevantes del conjunto limpio:
- Helpfulness_num: número de votos de utilidad que recibió la reseña,
- Length_summary: longitud del resumen como aproximación del nivel de detalle,
- user_group_detailed: transformada a índice numérico con StringIndexer para representar experiencia del usuario en la plataforma.

En modelos de aprendizaje no supervisado como KMeans, no hablamos de "precisión" como en los modelos supervisados, porque no se tiene una variable objetivo conocida con la cual comparar los resultado, en este caso la métricas de evaluación fue el Silhouette Score, la cual tuvo un valor resultantefue 0.6503, lo que indica una segmentación aceptable con buena cohesión interna y separación entre grupos.

